In [ ]:
# Step 1: Import Libraries and Load the Model
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import load_model


# Load the IMDB dataset word index
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}

# Load the pre-trained model with ReLU activation
model = load_model('simple_rnn_imdb.h5')
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (32, 500, 128)         │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (32, 128)              │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (32, 1)                │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,027 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

model.get_weights() gives you the weights applied between different parts of the RNN (input and hidden layers, hidden to hidden).

In [4]:
model.get_weights()

[array([[ 1.0012314e+00, -3.6222693e-01,  2.3774633e-01, ...,
         -2.3111078e-01, -3.9494547e-01,  7.4680167e-01],
        [-2.1049983e-03, -4.0705487e-02,  4.0687189e-02, ...,
          9.4483890e-05, -5.1462753e-03, -9.1155926e-03],
        [ 6.1172117e-02,  1.6845524e-01, -5.9356019e-02, ...,
          1.4126170e-01,  1.6356200e-01,  1.4277092e-01],
        ...,
        [ 2.7066046e-02, -9.4422251e-02,  4.0203113e-02, ...,
         -9.3202844e-02, -5.1095355e-02, -5.7722773e-02],
        [-3.1546220e-02, -9.3015745e-02,  3.5207395e-02, ...,
         -2.6807457e-02, -8.6943932e-02, -9.7819887e-02],
        [-1.1748834e-02, -8.2246937e-02,  9.2886068e-02, ...,
         -3.5567388e-02, -8.5530043e-02, -5.4510519e-02]], dtype=float32),
 array([[-0.04712019, -0.16543569, -0.14066234, ..., -0.04687752,
          0.0703236 ,  0.09358411],
        [-0.00598297,  0.00740785,  0.1272256 , ...,  0.08261929,
         -0.02037367, -0.02123228],
        [ 0.04363134,  0.11989255,  0.01447165

In [ ]:
# Step 2: Helper Functions
# Function to decode reviews
def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review])

# Function to preprocess user input
def preprocess_text(text):
    words = text.lower().split()    #.split() is used to split a string into a list of words 
    encoded_review = [word_index.get(word, 2) + 3 for word in words]  #word -> numbers(word index)
    padded_review = sequence.pad_sequences([encoded_review], maxlen=500)
    return padded_review

### model.predict()
→ Takes these word indices, passes them through layers:

Embedding layer: Turns each word index into a vector.

RNN layer: Processes the sequence of vectors to understand context.

Dense layer: Outputs a single number (probability of being positive).

In [ ]:
### Prediction  function

def predict_sentiment(review):
    preprocessed_input=preprocess_text(review)  #it turns the input text into a fixed-length list of word indices

    prediction=model.predict(preprocessed_input)

    sentiment = 'Positive' if prediction[0][0] > 0.5 else 'Negative'
    
    return sentiment, prediction[0][0]



Why prediction[0][0] instead of just [0]?
The model output shape is (1, 1) because:

You're passing 1 review (batch size = 1).

The final layer is Dense(1) → gives 1 probability per review.

So, the output looks like: [[0.92]]

prediction[0] → [0.92] (still a list/array)

prediction[0][0] → 0.92 (actual scalar value)

In [7]:
# Step 4: User Input and Prediction
# Example review for prediction
example_review = "This movie was fantastic! The acting was great and the plot was thrilling."

sentiment,score=predict_sentiment(example_review)

print(f'Review: {example_review}')
print(f'Sentiment: {sentiment}')
print(f'Prediction Score: {score}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
Review: This movie was fantastic! The acting was great and the plot was thrilling.
Sentiment: Positive
Prediction Score: 0.6871834993362427
